## How to open geoparquets

In [29]:
import geopandas as gpd
import s3fs
import pandas as pd
import boto3
import dask_geopandas
import dask.dataframe as dd
import matplotlib.pyplot as plt
import os

In [2]:
fs = s3fs.S3FileSystem()
bucket = 'ca-climate-index'
path = '2b_reproject/' 
pqt_list = [
    'climate_risk/flood/exposure/koordinates/climate_koordinates_floodplain.parquet.gzip',
    'climate_risk/wildfire/exposure/historical/iowa_state_environmental_mesonet/climate_iowa_mesonet_wildfire_warnings.parquet.gzip',
    'governance/community_preparedness/usda_forest_service/governance_usda_fuel_reduction.parquet.gzip',
    'governance/natural_resource_conservation/usda/forest_to_faucets/F2F2_Assessment/governance_usda_watershed_risk.parquet.gzip'
]

In [ ]:
for pqt in pqt_list:
    ppath = path+pqt
    bucket_uri = f's3://{bucket}/{ppath}'
    print(pqt)
    df = gpd.read_parquet(bucket_uri)
    display(df)